In [99]:
import torch
import cv2
import numpy as np

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [101]:
def create_gradient_mask(height, width, padding_vertical, padding_horizontal, padding_mode="mirror"):
    mask = np.ones((height, width), dtype=np.float32)
    
    if padding_vertical > 0:
        for i in range(padding_vertical):
            if padding_mode == "color":
                grad_value = (i / padding_vertical) ** 1.5
            else:
                grad_value = (i / padding_vertical) ** 2
            mask[i, :] = grad_value
            mask[height - 1 - i, :] = grad_value

    if padding_horizontal > 0:
        for i in range(padding_horizontal):
            if padding_mode == "color":
                grad_value = (i / padding_horizontal) ** 1.5
            else:
                grad_value = (i / padding_horizontal) ** 2
            mask[:, i] = grad_value
            mask[:, width - 1 - i] = grad_value

    return mask

In [102]:
def get_pastel_color(image):
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    height, width, _ = image.shape
    center_region = hsv_img[height//4:3*height//4, width//4:3*width//4]
    avg_color_hsv = np.mean(center_region, axis=(0, 1))

    avg_color_hsv[1] *= 0.3
    avg_color_hsv[2] = min(avg_color_hsv[2] * 1.5, 255)

    pastel_color = cv2.cvtColor(np.uint8([[avg_color_hsv]]), cv2.COLOR_HSV2BGR)[0][0]
    return tuple(int(c) for c in pastel_color)

In [103]:
def add_blurred_padding(image_path, blur_strength=25, sigma=0, padding_mode="mirror"):
    img = cv2.imread(image_path)

    height, width, channels = img.shape

    if height > width:
        padding_vertical = 0
        padding_horizontal = (height - width) // 2
    else:
        padding_vertical = (width - height) // 2
        padding_horizontal = 0

    if padding_mode == "mirror":
        padded_img = cv2.copyMakeBorder(
            img, 
            padding_vertical, padding_vertical, padding_horizontal, padding_horizontal, 
            borderType=cv2.BORDER_REFLECT
        )
    elif padding_mode == "color":
        pastel_color = get_pastel_color(img)

        padded_img = cv2.copyMakeBorder(
            img, 
            padding_vertical, padding_vertical, padding_horizontal, padding_horizontal, 
            borderType=cv2.BORDER_CONSTANT, 
            value=pastel_color
        )

    blurred_img = cv2.GaussianBlur(padded_img, (blur_strength, blur_strength), sigma)

    mask = create_gradient_mask(padded_img.shape[0], padded_img.shape[1], padding_vertical, padding_horizontal, padding_mode)

    mask_3d = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
    blended_img = (mask_3d * padded_img + (1 - mask_3d) * blurred_img).astype(np.uint8)

    return blended_img

In [104]:
def show_image(image):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    cv2.imshow("Padded Image", img_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
image_path = "Lay of the Land.png"
padded_image = add_blurred_padding(image_path, 51, 30, padding_mode="color")
cv2.imwrite("padded_output.png", padded_image)